# Real Time Detection - Without PCB Integration

In [ ]:
import os
from ultralytics import YOLO
import cv2

# Set up the video capture for the webcam
cap = cv2.VideoCapture(0)  # 0 is the default camera

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Path to the trained model
model_path = os.path.join('.', 'weights', 'last.pt')

# Load the custom model
model = YOLO(model_path)

threshold = 0.5

# Get the screen dimensions
screen_width = 1280  # Replace with your screen width
screen_height = 720  # Replace with your screen height

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame from webcam.")
        break

    results = model(frame)[0]

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        if score > threshold:
            label = results.names[int(class_id)].upper()
            color = (0, 255, 0) if label != "WRONG" else (0, 0, 255)  # Green for correct, red for wrong

            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 4)
            cv2.putText(frame, label, (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, color, 3, cv2.LINE_AA)

    # Resize the frame to fit within the screen dimensions
    frame_height, frame_width = frame.shape[:2]
    aspect_ratio = frame_width / frame_height

    if frame_width > screen_width or frame_height > screen_height:
        if aspect_ratio > 1:
            new_width = screen_width
            new_height = int(new_width / aspect_ratio)
        else:
            new_height = screen_height
            new_width = int(new_height * aspect_ratio)

        frame = cv2.resize(frame, (new_width, new_height))
    else:
        frame = cv2.resize(frame, (screen_width, screen_height))

    # Display the frame with detection boxes
    cv2.imshow('YOLO Real-Time Detection', frame)

    # Press 'q' to exit the real-time detection
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

# Real Time Detection - With PCB Integration

In [ ]:
import os
from ultralytics import YOLO
import cv2
import serial
import time

# Initialize Arduino serial communication
arduino_port = 'COM3'
baud_rate = 9600
ser = serial.Serial(arduino_port, baud_rate)
time.sleep(2)  # Wait for the serial connection to initialize

# Set up the video capture for the webcam
cap = cv2.VideoCapture(0)  # 0 is the default camera

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Path to the trained model
model_path = os.path.join('.', 'weights', 'last.pt')

# Load the custom model
model = YOLO(model_path)

threshold = 0.5

# Initialize detection tracking variables
wrong_detected = False
correct_count = 0
start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame from webcam.")
        break

    if not wrong_detected:
        results = model(frame)[0]
        
        for result in results.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = result

            if score > threshold:
                label = results.names[int(class_id)].upper()
                color = (0, 255, 0) if label != "WRONG" else (0, 0, 255)
                if label == "WRONG":
                    wrong_detected = True

                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 4)
                cv2.putText(frame, label, (int(x1), int(y1 - 10)),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.3, color, 3, cv2.LINE_AA)

        # If wrong label detected, reset correct count and start tracking time
        if wrong_detected:
            correct_count = 0
            start_time = time.time()

    else:
        # Count correct detections for 16 seconds after wrong detection
        if time.time() - start_time <= 16:
            results = model(frame)[0]

            for result in results.boxes.data.tolist():
                x1, y1, x2, y2, score, class_id = result

                if score > threshold:
                    label = results.names[int(class_id)].upper()
                    color = (0, 255, 0) if label != "WRONG" else (0, 0, 255)
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 4)
                    cv2.putText(frame, label, (int(x1), int(y1 - 10)),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.3, color, 3, cv2.LINE_AA)

                    
                    if label == "CORRECT":
                        correct_count += 1

        else:
            # Decision based on correct count after 16 seconds
            if correct_count >= 5:
                # Do not trigger pushing mechanism
                print("No action taken. Correct detections were sufficient.")
            else:
                # Trigger pushing mechanism
                ser.write(b'W')
                print("Pushing mechanism triggered.")

            # Reset variables for next detection cycle
            wrong_detected = False
            correct_count = 0

    # Display the frame with detection boxes
    cv2.imshow('YOLO Real-Time Detection', frame)

    # Press 'q' to exit the real-time detection
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
ser.close()
cv2.destroyAllWindows()